# Практическая работа

# Задача

In [2]:
import pandas as pd
import numpy as np
import pickle

# from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


## Задание 1

В предыдущем задании вы собрали всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

Загрузите эту таблицу.

In [98]:
df = pd.read_pickle('df_clean.p')

Посмотрим на распределение категориальных и числовых столбцов:

In [35]:
df.dtypes.value_counts()

int64      9
object     7
float64    7
Name: count, dtype: int64

In [44]:
df.GEN_INDUSTRY.value_counts()

GEN_INDUSTRY
Торговля                                     2385
Другие сферы                                 1709
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт                                     787
Сельское хозяйство                            702
Строительство                                 573
Коммунальное хоз-во/Дорожные службы           533
Ресторанный бизнес/Общественное питание       408
Наука                                         403
Нефтегазовая промышленность                   225
Сборочные производства                        172
Банк/Финансы                                  169
Энергетика                                    145
Развлечения/Искусство                         141
ЧОП/Детективная д-ть                          136
Информационные услуги                         108
Салоны красоты и здоровья            

In [45]:
df.GEN_TITLE.value_counts()

GEN_TITLE
Специалист                        7009
Рабочий                           3075
Служащий                           904
Руководитель среднего звена        697
Работник сферы услуг               563
Высококвалифиц. специалист         549
Руководитель высшего звена         427
Индивидуальный предприниматель     217
Другое                             177
Руководитель низшего звена         136
Военнослужащий по контракту         88
Партнер                             13
Name: count, dtype: int64

In [47]:
df.FAMILY_INCOME.value_counts()

FAMILY_INCOME
от 10000 до 20000 руб.    6339
от 20000 до 50000 руб.    5882
от 5000 до 10000 руб.     1121
свыше 50000 руб.           486
до 5000 руб.                27
Name: count, dtype: int64

Для обучения линейных моделей категориальные столбцы придётся закодировать. Не планирую заморачиваться с кодировкой, буду использовать обычный OHE 

In [41]:
# df = df.astype({'GENDER': 'bool', 'DEPENDANTS': 'bool', 'SOCSTATUS_WORK_FL': 'bool', 'SOCSTATUS_PENS_FL': 'bool', 'FL_PRESENCE_FL': 'bool', 'OWN_AUTO': 'bool', 'TARGET': 'bool'})

## Логистическая регрессия (с параметрами по умолчанию), настройка порогов для оптимизации метрик

На тренировочных данных обучите линейную модель классификации для предсказания целевой переменной (столбец `TARGET`).

Сделайте прогноз вероятности отклика на рекламную кампанию для тестовых данных.

Разбейте данные на тренировочную и тестовую часть в пропорции 80% к 20%, зафиксируйте `random_state = 42`.

У нас есть два столбца, которые по сути представляют собой уникальный индентификатор - id заёщика (ID) и id кредитного договора (AGREEMENT_RK). Для обучения эти данные выглядят бесполезные. С учетом того, что нам предстоит предсказать поведение клиента, я удалю id кредитного договора, а id заёмщика сделаю индексом, через который впоследствие будет удобно обращаться к данным пользователя для индивидуального прогноза или идентификации результатов.

In [227]:
X = df.drop(columns = ['TARGET', 'AGREEMENT_RK']).reset_index(drop=True).set_index('ID')
y = df.TARGET

In [ ]:
pd.to_pickle(X, 'X.p', compression='gzip')
pd.to_pickle(y, 'y.p', compression='gzip')

In [101]:
X.sample()

,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,FACT_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO,CREDIT,TERM,FST_PAYMENT,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME
ID,,,,,,,,,,,,,,,,,,,,
106817880,31,1,Высшее,Состою в браке,1,1,1,0,Нижегородская область,0,0,3000.0,3.0,690.95,Образование,Рабочий,Участие в основ. деятельности,12.0,от 20000 до 50000 руб.,5000.0


In [102]:
y.sample()

14368    0.0
Name: TARGET, dtype: float64

In [103]:
y.value_counts()

TARGET
0.0    12093
1.0     1762
Name: count, dtype: int64

Наши данные сильно несбалансированные. Для обучения моделей есть смысл использовать параметр stratify, чтобы разбивка на трейн и тест была происходила пропорционально классам в датасете 

In [238]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [206]:
y_test

13033    0.0
7958     0.0
4392     0.0
6484     0.0
12648    0.0
        ... 
10952    1.0
15596    0.0
1132     0.0
545      0.0
8642     0.0
Name: TARGET, Length: 2771, dtype: float64

В следующих четырёх строчках кода ниже, я пробовал облегчить себе задачу и провернуть всё в едином пайплайне. Однако постоянно сталкивался с разным набором ошибок (то памяти не хватало, то ему не нравилась спарс-матрицы, то категориальные столбцы не очень дружат в реализацией OHE в sklearn. В итоге пришлось пойти длинным путём и использовать стороннюю библиотеку

In [239]:
columns = X_train.select_dtypes('object').columns
columns

Index(['EDUCATION', 'MARITAL_STATUS', 'FACT_ADDRESS_PROVINCE', 'GEN_INDUSTRY',
       'GEN_TITLE', 'JOB_DIR', 'FAMILY_INCOME'],
      dtype='object')

In [230]:
# from sklearn.preprocessing import OneHotEncoder

In [126]:
pipe = Pipeline([
    ('encoder', OneHotEncoder(categories=list(columns), drop='first', sparse_output=False)),
    ('scaler', StandardScaler()),
    ('features', PolynomialFeatures(degree=2)),
    ('model', LogisticRegression())
    ])

In [124]:
pipe.fit(X_train, y_train)

pred_pipe = pipe.predict(X_test)
recall_score(y_test, pred_pipe)

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).

Итак, кодируем сторонней библиотекой

In [37]:
X_train.shape, X_test.shape

((11084, 20), (2771, 20))

In [240]:
from category_encoders.one_hot import OneHotEncoder
ohe_enc = OneHotEncoder(cols=columns)
X_train = pd.DataFrame(ohe_enc.fit_transform(X_train), index=X_train.index)
X_test = pd.DataFrame(ohe_enc.transform(X_test), index=X_test.index)

In [241]:
X_train.shape, X_test.shape

((11084, 164), (2771, 164))

In [242]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns =X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns =X_test.columns, index=X_test.index)

In [243]:
X_train.sample()

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
ID,,,,,,,,,,,,,,,,,,,,,
106818368,0.399082,0.753345,-0.868535,-0.526828,1.536498,-0.197965,-0.128119,-0.009499,-0.035562,-0.561929,...,-0.123682,-0.026875,-0.066636,-0.00163,1.095282,-0.861839,-0.192486,-0.297816,-0.044596,-0.469906


In [151]:
X_train.shape, X_test.shape

((11084, 164), (2771, 164))

In [152]:
model = LogisticRegression()

In [153]:
model.fit(X_train, y_train)

LogisticRegression()

In [154]:
y_pred = model.predict(X_test)

In [155]:
recall_score(y_test, y_pred)

0.002840909090909091

In [156]:
accuracy_score(y_test, y_pred)

0.8715265247203176

Переведите вероятности в классы по стандартному порогу (0.5) и на тестовом наборе данных вычислите метрики:

* accuracy
* precision
* recall
* f1-score

In [65]:
y.value_counts()

TARGET
0.0    12093
1.0     1762
Name: count, dtype: int64

In [158]:
y_prob = model.predict_proba(X_test)[:,1]

In [159]:
y_prob

array([0.07741067, 0.08188389, 0.20035837, ..., 0.3687718 , 0.143651  ,
       0.04691201])

In [183]:
y_trhold = y_prob > 0.5
accuracy = accuracy_score(y_test, y_trhold)
precision = precision_score(y_test, y_trhold)
recall = recall_score(y_test, y_trhold)
f1 = f1_score(y_test, y_trhold)
print(f'{accuracy=}, {precision=}, {recall=}, {f1=}')

accuracy=0.8715265247203176, precision=0.16666666666666666, recall=0.002840909090909091, f1=0.00558659217877095


In [182]:
y_trhold = y_prob > 0.10
accuracy = accuracy_score(y_test, y_trhold)
precision = precision_score(y_test, y_trhold)
recall = recall_score(y_test, y_trhold)
f1 = f1_score(y_test, y_trhold)
print(f'{accuracy=}, {precision=}, {recall=}, {f1=}')

accuracy=0.4926019487549621, precision=0.15690104166666666, recall=0.6846590909090909, f1=0.2552966101694915


Целевая метрика для задачи - полнота, так как нам нужно найти максимум клиентов, кто может откликнуться на рекламу.

Но при этом точность не должна просесть, поэтому за ней тоже следим.

Разбейте тренировочные данные на `train` и `val` части в пропорции 3 к 1.

В цикле:

* переберите пороги от 0 до 1 с шагом 0.01
* вычислите для каждого порога значение метрик precision и recall
* подберите такой порог, при котором recall не меньше 0.66, а точность максимальна.

In [304]:
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train, test_size=1/3, stratify=y_train)

Xtrain.shape, Xval.shape

((7389, 20), (3695, 20))

In [316]:
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train, test_size=1/3, stratify=y_train)

num_cols = X_train.select_dtypes(include=np.number).columns
cat_cols = X_train.select_dtypes(exclude=np.number).columns

trans = make_column_transformer((StandardScaler(), num_cols), 
                            (OneHotEncoder(), cat_cols), remainder='passthrough')

val_model = Pipeline([
    ('encoder', trans),
    ('features', PolynomialFeatures(degree=2)),
    ('model', LogisticRegression(max_iter=1000))
])
val_model.fit(Xtrain, ytrain)
pred_probability = val_model.predict_proba(Xval)[:, 1]

In [306]:
val_model.fit(Xtrain, ytrain)

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  Index(['AGE', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL',
       'SOCSTATUS_PENS_FL', 'FL_PRESENCE_FL', 'OWN_AUTO', 'CREDIT', 'TERM',
       'FST_PAYMENT', 'WORK_TIME', 'PERSONAL_INCOME'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  Index(['EDUCATION', 'MARITAL_STATUS', 'FACT_ADDRESS_PROVINCE', 'GEN_INDUSTRY',
       'GEN_TITLE', 'JOB_DIR', 'FAMILY_INCOME'],
      dtype='object'))])),
                ('features', PolynomialFeatures()),
                ('model', LogisticRegression(max_iter=1000))])

In [317]:
pred_probability

array([0.0619966 , 0.00187708, 0.3838538 , ..., 0.02408115, 0.01138025,
       0.02962157])

In [322]:
BestRec = -1
BestThr = -1
Acc_max = -1

for thr in np.arange(0, 1, 0.01):
    pred_class = pred_probability >= thr
    new_recall = recall_score(yval, pred_class)
    new_accuracy = accuracy_score(yval, pred_class)
    precision = precision_score(yval, pred_class)
    print(f'{thr=}, {new_recall=}, {precision=}')
    if new_accuracy >= Acc_max and new_recall > 0.66:
      BestRec = new_recall
      BestThr = thr
      Acc_max = new_accuracy
print (f'\nЛучший результат: {BestThr=}, {BestRec=}, {Acc_max=}')

thr=0.0, new_recall=1.0, precision=0.12719891745602166
thr=0.01, new_recall=0.7957446808510639, precision=0.13856984068173397
thr=0.02, new_recall=0.7106382978723405, precision=0.14604285089637078
thr=0.03, new_recall=0.6638297872340425, precision=0.15339233038348082
thr=0.04, new_recall=0.6127659574468085, precision=0.15542363734484618
thr=0.05, new_recall=0.5765957446808511, precision=0.1602602010644589
thr=0.06, new_recall=0.548936170212766, precision=0.1634980988593156
thr=0.07, new_recall=0.5255319148936171, precision=0.16689189189189188
thr=0.08, new_recall=0.4957446808510638, precision=0.16642857142857143
thr=0.09, new_recall=0.4765957446808511, precision=0.1691842900302115
thr=0.1, new_recall=0.4595744680851064, precision=0.17170111287758347
thr=0.11, new_recall=0.42340425531914894, precision=0.16569525395503748
thr=0.12, new_recall=0.4021276595744681, precision=0.16535433070866143
thr=0.13, new_recall=0.3872340425531915, precision=0.16743330266789327
thr=0.14, new_recall=0.355

Для выбранного порога посчитайте все метрики на тестовых данных. Сильно ли они отличаются от метрик на валидации?

 Заново обучу модель на исходных тренировочных данных (`X_train`, `y_train`), и предскажу вероятности на тесте и переведу их в классы по найденному порогу.

In [323]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Долго мучился, но кажется победил работу с пайплайном. Так, конечно, проще, но часто возникают неочевидные проблемы из-за OHE кодировщика

In [324]:
trans = make_column_transformer((StandardScaler(), num_cols), 
                            (OneHotEncoder(), cat_cols))

pipe = Pipeline([
    ('encoder', trans),
    ('features', PolynomialFeatures(degree=2)),
    ('model', LogisticRegression(max_iter=1000))
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('encoder',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  Index(['AGE', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL',
       'SOCSTATUS_PENS_FL', 'FL_PRESENCE_FL', 'OWN_AUTO', 'CREDIT', 'TERM',
       'FST_PAYMENT', 'WORK_TIME', 'PERSONAL_INCOME'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  Index(['EDUCATION', 'MARITAL_STATUS', 'FACT_ADDRESS_PROVINCE', 'GEN_INDUSTRY',
       'GEN_TITLE', 'JOB_DIR', 'FAMILY_INCOME'],
      dtype='object'))])),
                ('features', PolynomialFeatures()),
                ('model', LogisticRegression(max_iter=1000))])

In [325]:
pred_probability = pipe.predict_proba(X_test)[:, 1]

In [327]:
y_trhold = pred_probability > BestThr
accuracy = accuracy_score(y_test, y_trhold)
precision = precision_score(y_test, y_trhold)
recall = recall_score(y_test, y_trhold)
f1 = f1_score(y_test, y_trhold)
print(f'{accuracy=}, {precision=}, {recall=}, {f1=}')

accuracy=0.43305665824612055, precision=0.13568439928272563, recall=0.6448863636363636, f1=0.22419753086419755


In [352]:
confusion_matrix(y_test, y_trhold)

array([[ 973, 1446],
       [ 125,  227]], dtype=int64)

Результаты на тесте отличаются от результатов на валидации. и recall и accuracy немного просели, однако Уровень recall остался не неплохом уровне в 0.64.
Сами по себе уровни precision и accuracy мне кажутся довольно низкими: при заданном уровне recall нашей маркетинговой службе придётся сделать довольно много лишней работы в отношении тех клиентов, которые не согласятся на сотрудничество. При этом 1/3 потенциальных клиентов, которые могли бы сотрудничать с банком, будут упущены в списках и не отработаны. 

Выведите на экран в виде таблицы топ-6 признаков с наибольшими по модулю весами модели.

In [332]:
pipe[:-1].get_feature_names_out()

array(['1', 'standardscaler__AGE', 'standardscaler__GENDER', ...,
       'onehotencoder__FAMILY_INCOME_от 5000 до 10000 руб.^2',
       'onehotencoder__FAMILY_INCOME_от 5000 до 10000 руб. onehotencoder__FAMILY_INCOME_свыше 50000 руб.',
       'onehotencoder__FAMILY_INCOME_свыше 50000 руб.^2'], dtype=object)

In [342]:
pipe[-1].coef_

array([[ 0.00139845, -0.19203255,  0.0411108 , ..., -0.08234983,
         0.        ,  0.17765374]])

In [344]:
pipe[1].get_feature_names_out()

array(['1', 'x0', 'x1', ..., 'x162^2', 'x162 x163', 'x163^2'],
      dtype=object)

С учётом того, что я использовал полиномиальные фичи и пайплайн, мне сложно проанализировать веса: признаков очень много, и кодировщики зашифровали названия столбцов не вполне очевидно.
В целом мне кажется, что использование полиномиальных фичей было возможно слишком избыточной штукой. Но упрощять и проверять эту теорию мне уже лениво. Сохраним прогнозы и обученную модель

In [345]:
X_test['predictions'] = pred_probability

X_test[['predictions']].to_csv("PredictionsChurn.csv", index=False)

In [346]:
with open('model.pickle', 'wb') as f:
    pickle.dump(pipe, f)

In [347]:
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)

In [350]:
X_test.iloc[1,].values  # Так будем выводить сведения об объекте

array([56, 1, 'Высшее', 'Не состоял в браке', 2, 0, 1, 1,
       'Иркутская область', 0, 0, 12550.0, 10.0, 3200.0, 'Строительство',
       'Другое', 'Вспомогательный техперсонал', 36.0,
       'от 10000 до 20000 руб.', 16000.0, 0.01077674597751144],
      dtype=object)

In [349]:
model.predict_proba(X_test.iloc[486,:].to_frame().T)[:,1] > BestThr  # а так будем получать предсказания

array([False])

## Лог.регрессия и SVM, регуляризация

В этот раз обойдусь без полиномиальных фичей

In [253]:
X = pd.read_pickle('X.p', compression='gzip')
y = pd.read_pickle('y.p', compression='gzip')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [7]:
import pickle

with open('ohe.pickle', 'rb') as f:
    ohe_enc = pickle.load(f)
    
with open('scaler.pickle', 'rb') as f:
    scaler = pickle.load(f)

In [254]:
X_train = pd.DataFrame(ohe_enc.transform(X_train), index=X_train.index)
X_test = pd.DataFrame(ohe_enc.transform(X_test), index=X_test.index)
X_train = pd.DataFrame(scaler.transform(X_train), columns =X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns =X_test.columns, index=X_test.index)

In [9]:
X_test.head()

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
ID,,,,,,,,,,,,,,,,,,,,,
106808150,-0.638438,0.754228,-0.863430,1.888638,-0.651252,-0.199436,-0.128482,-0.039193,-0.009499,0.776461,...,-0.186871,-0.075001,-0.026875,-0.007783,1.087938,-0.858664,-0.189185,-0.296558,-0.047546,-0.714982
106810986,1.607043,0.754228,-0.863430,1.888638,-0.651252,-0.199436,-0.128482,-0.039193,-0.009499,-1.287894,...,-0.186871,-0.075001,-0.026875,-0.011307,1.087938,-0.858664,-0.189185,-0.296558,-0.047546,0.178995
106807679,-1.199808,-1.325858,1.158171,-0.529482,-0.651252,-0.199436,-0.128482,-0.039193,-0.009499,0.776461,...,-0.186871,-0.075001,-0.026875,-0.012409,-0.919170,1.164600,-0.189185,-0.296558,-0.047546,0.290742
106816628,0.390741,0.754228,1.158171,-0.529482,-0.651252,-0.199436,-0.128482,-0.039193,-0.009499,0.776461,...,-0.186871,-0.075001,-0.026875,-0.010426,1.087938,-0.858664,-0.189185,-0.296558,-0.047546,-0.759680
106807602,1.700605,0.754228,-0.863430,-0.529482,1.535503,-0.199436,-0.128482,-0.039193,-0.009499,-1.287894,...,-0.186871,-0.075001,-0.026875,-0.006461,-0.919170,1.164600,-0.189185,-0.296558,-0.047546,0.625983


In [430]:
trans = make_column_transformer((StandardScaler(), num_cols), 
                            (OneHotEncoder(), cat_cols))

modelLR = LogisticRegression(max_iter=1000)
modelSVM = SVC(kernel='linear')

pipe_LR = Pipeline([
    ('encoder', trans),
    ('model', modelLR)
])

pipe_SVM = Pipeline([
    ('encoder', trans),
    ('model', modelSVM)
])

C = np.arange(1, 10, 0.1)
params = {'C': C}
# params = {'model__C': C}

In [422]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# gs_model_linear = GridSearchCV(estimator=pipe_LR, param_grid=params, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1, error_score='raise')
# gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)
# gs_model_linear.fit(X_train, y_train)

In [435]:
ohe_enc = OneHotEncoder()
X_train = pd.DataFrame(ohe_enc.fit_transform(X_train, drop='first'), index=X_train.index)
X_test = pd.DataFrame(ohe_enc.transform(X_test), index=X_test.index)

In [436]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns =X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns =X_test.columns, index=X_test.index)

In [421]:
X_test.iloc[5].to_frame().T

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
106813158,0.85855,0.754228,-0.86343,-0.529482,1.535503,-0.199436,-0.128482,-0.039193,-0.009499,0.776461,...,-0.186871,-0.075001,-0.026875,-0.005287,1.087938,-0.858664,-0.189185,-0.296558,-0.047546,-1.050223


In [425]:
ohe_enc.transform(X_test.iloc[5].to_frame().T)

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
106813737,23,1,0,1,0,0,0,0,0,0,...,0,0,0,36.0,1,0,0,0,0,11000.0


In [432]:
modelLR = LogisticRegression(max_iter=1000)

In [433]:
gs_model_linear = GridSearchCV(estimator=modelLR, param_grid=params, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1, error_score='raise')

In [437]:
gs_model_linear.fit(X_train, y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


GridSearchCV(cv=3, error_score='raise',
             estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': array([1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2,
       2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5,
       3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8,
       4.9, 5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1,
       6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4,
       7.5, 7.6, 7.7, 7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7,
       8.8, 8.9, 9. , 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='roc_auc', verbose=1)

In [443]:
gs_model_linear.best_params_

{'C': 1.2000000000000002}

Лучшая регуляризация для линейного ядра - С=1.2 

In [77]:
modelLR = LogisticRegression(max_iter=1000, C=1.2).fit(X_train, y_train)

In [80]:
y_pred = modelLR.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

0.6166252489759104

Итак, для настроенной логистической регрессии ROC-AUC составил 0.617

### Настройка параметров метода опорных векторов

Пытаясь перебрать все необходимые параметры через ГридСёрч столкнулся с тем, что подбор идёт очень медленно: я не мог дождаться завершения работы скрипта и через 6 часов.

In [439]:
# params = {'kernel': ['rbf', 'linear', 'sigmoid'], 'C': [1, 10, 100]}
# C = np.arange(1, 10, 0.1)
# params = {'C': C}
# 
# modelSVM = SVC(kernel='linear')
# gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)
# gs_model_SVM.fit(X_train, y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


KeyboardInterrupt: 

В итоге я понял, что нужно действовать мелкими итерациями: сначала выбрать оптимальное ядро, потом по логарифмической шкале отобрать уровень регуляризации, потом в рамках этого уровня подбирать интервалами. Так что ниже много однотипного кода, на каждой из итерациях которого параметры метода всё более сходились к необходимому мне оптимуму метрики roc-auc

In [10]:
# Выбираем ядро
params = {'kernel': ['rbf', 'linear', 'sigmoid']}
modelSVM = SVC()
gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=True, n_jobs=-1)
gs_model_SVM.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'kernel': ['rbf', 'linear', 'sigmoid']},
             scoring='roc_auc', verbose=True)

In [11]:
gs_model_SVM.best_params_

{'kernel': 'rbf'}

In [12]:
# Выбираем порядок регуляризации логарифмической шкалой
params = {'C': [1, 10, 100]}

In [13]:
modelSVM = SVC(kernel='rbf')
gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=True, n_jobs=-1)
gs_model_SVM.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1, param_grid={'C': [1, 10, 100]},
             scoring='roc_auc', verbose=True)

In [14]:
gs_model_SVM.best_params_

{'C': 1}

In [16]:
modelSVM = SVC(kernel='rbf', C=1)
modelSVM.fit(X_train, y_train)

SVC(C=1)

In [30]:
# определяем диапазон внутри десятков
C = np.arange(1, 11, 3)
params = {'C': C}
modelSVM = SVC(kernel='rbf')
gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=True, n_jobs=-1)
gs_model_SVM.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([ 1,  4,  7, 10])}, scoring='roc_auc',
             verbose=True)

In [31]:
gs_model_SVM.best_params_

{'C': 4}

In [34]:
# уточняем диапазон вокруг четвёрки
C = np.arange(2, 7, 1)
params = {'C': C}
modelSVM = SVC(kernel='rbf')
gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=True, n_jobs=-1)
gs_model_SVM.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([2, 3, 4, 5, 6])}, scoring='roc_auc',
             verbose=True)

In [35]:
gs_model_SVM.best_params_

{'C': 3}

In [36]:
# последняя итерация: настраиваем диапазон до десятых
C = np.arange(2.1, 4, 0.1)
params = {'C': C}
modelSVM = SVC(kernel='rbf')
gs_model_SVM = GridSearchCV(estimator=modelSVM, param_grid=params, cv=3, scoring='roc_auc', verbose=True, n_jobs=-1)
gs_model_SVM.fit(X_train, y_train)

Fitting 3 folds for each of 19 candidates, totalling 57 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3,
       3.4, 3.5, 3.6, 3.7, 3.8, 3.9])},
             scoring='roc_auc', verbose=True)

In [37]:
gs_model_SVM.best_params_

{'C': 3.300000000000001}

Итак, лучшая регуляризация для ядра С=3.3, лучшее ядро - rbf

In [40]:
modelSVM = SVC(kernel='rbf', probability=True, C=3.3)
modelSVM.fit(X_train, y_train)

SVC(C=3.3, probability=True)

In [43]:
y_pred = modelSVM.predict_proba(X_test)[:,1]

In [44]:
roc_auc_score(y_test, y_pred)

0.5409119095043031

Итоговый показатель roc-auc метода опорным векторов на лучших параметрах даёт худший roc-auc по сравнению с линейной регрессией. Для приложения будем использовать её

## Задание 2

Добавьте в Streamlit-приложение визуализацию результатов модели:

* опцию выбора порога и вывод метрик качества в зависимости от выбранного порога

* вывод прогноза модели на выбранном объекте (клиенте) - вероятность отклика на рекламу.

## Сборка необходимых функций для Strimlit 

In [82]:
def get_metrics_score(y_test, y_pred)->dict:
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [71]:
# прогнозы для лучшей модели SVC c подобранным порогом
y_pred = modelSVM.predict(X_test)
get_metrics_score(y_test=y_test, y_pred=y_pred)

{'accuracy': 0.8661133164922411,
 'precision': 0.14814814814814814,
 'recall': 0.011363636363636364,
 'f1': 0.02110817941952507}

In [81]:
# прогнозы для логистической регрессии с подобранным порогом
y_pred = modelLR.predict(X_test)
get_metrics_score(y_test=y_test, y_pred=y_pred)

{'accuracy': 0.8715265247203176,
 'precision': 0.16666666666666666,
 'recall': 0.002840909090909091,
 'f1': 0.00558659217877095}

In [97]:
def get_metrics_score_thr(y_test, pred_probability, threshold: float)-> dict:
    y_thr = pred_probability > threshold
    return get_metrics_score(y_test, y_thr)    

In [207]:
preds = pd.read_csv('PredictionsChurn.csv')
len(preds)

2771

In [208]:
preds = pd.read_csv('Predictions_regular.csv')
len(preds)

13855

In [99]:
get_metrics_score_thr(y_test, preds, 0.1)

{'accuracy': 0.6282930350054132,
 'precision': 0.1526639344262295,
 'recall': 0.42329545454545453,
 'f1': 0.22439759036144577}

In [145]:
X.loc[106809308]

AGE                                                 28
GENDER                                               1
EDUCATION                          Среднее специальное
MARITAL_STATUS                          Состою в браке
CHILD_TOTAL                                          1
DEPENDANTS                                           1
SOCSTATUS_WORK_FL                                    1
SOCSTATUS_PENS_FL                                    0
FACT_ADDRESS_PROVINCE                Читинская область
FL_PRESENCE_FL                                       0
OWN_AUTO                                             0
CREDIT                                         19498.0
TERM                                              12.0
FST_PAYMENT                                        0.0
GEN_INDUSTRY                                  Торговля
GEN_TITLE                                   Специалист
JOB_DIR                  Участие в основ. деятельности
WORK_TIME                                          5.0
FAMILY_INC

In [95]:
X.sample()

,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,FACT_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO,CREDIT,TERM,FST_PAYMENT,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME
ID,,,,,,,,,,,,,,,,,,,,
106807684,38,1,Среднее специальное,Состою в браке,2,2,1,0,Алтайский край,0,0,7092.0,5.0,788.0,Наука,Специалист,Участие в основ. деятельности,18.0,от 10000 до 20000 руб.,9000.0


In [143]:
X.describe(include='object').loc['top']

EDUCATION                          Среднее специальное
MARITAL_STATUS                          Состою в браке
FACT_ADDRESS_PROVINCE              Кемеровская область
GEN_INDUSTRY                                  Торговля
GEN_TITLE                                   Специалист
JOB_DIR                  Участие в основ. деятельности
FAMILY_INCOME                   от 10000 до 20000 руб.
Name: top, dtype: object

In [144]:
X.describe().loc['mean']

AGE                     38.760159
GENDER                   0.639336
CHILD_TOTAL              1.086611
DEPENDANTS               0.696355
SOCSTATUS_WORK_FL        0.999350
SOCSTATUS_PENS_FL        0.049080
FL_PRESENCE_FL           0.308986
OWN_AUTO                 0.123782
CREDIT               14934.855620
TERM                     8.144497
FST_PAYMENT           3474.026995
WORK_TIME              292.211981
PERSONAL_INCOME      14413.438987
Name: mean, dtype: float64

106818876

Единичный прогноз

In [158]:
from random import choice
def get_single_pred(needed_index: bool = True, model_type:str = 'regular') -> float:
    # X = df.drop(columns = ['TARGET', 'AGREEMENT_RK']).reset_index(drop=True).set_index('ID')
    index = choice(X.index.tolist()) if needed_index else 0
    df = pd.DataFrame(ohe_enc.transform(X.loc[index].to_frame().T))
    df = pd.DataFrame(scaler.transform(df), columns =df.columns, index=df.index)
    model = model_regular if model_type == 'regular' else model_tuned
    single_pred_positive = model.predict_proba(df)[:,1][0] 
    return single_pred_positive

In [128]:
get_single_pred(0)

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
106805103,0.297179,0.754228,-0.86343,-0.529482,1.535503,-0.199436,-0.128482,-0.039193,-0.009499,-1.287894,...,-0.186871,-0.075001,-0.026875,-0.012519,-0.91917,1.1646,-0.189185,-0.296558,-0.047546,1.184718


In [126]:
final_df

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
106805103,0.297179,0.754228,-0.86343,-0.529482,1.535503,-0.199436,-0.128482,-0.039193,-0.009499,-1.287894,...,-0.186871,-0.075001,-0.026875,-0.012519,-0.91917,1.1646,-0.189185,-0.296558,-0.047546,1.184718


In [133]:
modelLR.predict_proba(final_df)[:,1][0]

0.1860422704584153

In [135]:
modelSVM.predict_proba(final_df)[:,1][0]

0.14715016836804687

In [195]:
with open('model_regular.pickle', 'rb') as f:
    model_regular = pickle.load(f)

In [196]:
model_regular

Pipeline(steps=[('encoder',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  Index(['AGE', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL',
       'SOCSTATUS_PENS_FL', 'FL_PRESENCE_FL', 'OWN_AUTO', 'CREDIT', 'TERM',
       'FST_PAYMENT', 'WORK_TIME', 'PERSONAL_INCOME'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  Index(['EDUCATION', 'MARITAL_STATUS', 'FACT_ADDRESS_PROVINCE', 'GEN_INDUSTRY',
       'GEN_TITLE', 'JOB_DIR', 'FAMILY_INCOME'],
      dtype='object'))])),
                ('features', PolynomialFeatures()),
                ('model', LogisticRegression(max_iter=1000))])

In [197]:
df = pd.read_pickle('df_clean.p')
X = df.drop(columns = ['TARGET', 'AGREEMENT_RK']).reset_index(drop=True).set_index('ID')

In [219]:
pred_probability = model_regular.predict_proba(X_test)[:,1]

In [220]:
pred_probability

array([0.01154493, 0.01077675, 0.0046805 , ..., 0.90248246, 0.11881112,
       0.30567806])

In [221]:
len(pred_probability)

2771

In [222]:
X_test['predictions'] = pred_probability

X_test[['predictions']].to_csv("Predictions_regular_test.csv", index=True)

In [223]:
X_test['predictions'].mean()

0.13000259593674407

In [224]:
modelLR

LogisticRegression(C=1.2, max_iter=1000)

In [225]:
with open('model_tuned.pickle', 'wb') as f:
    pickle.dump(modelLR, f)

In [296]:
with open('y_test.p', 'wb') as f:
    pickle.dump(y_test, f)

In [244]:
pred_probability = modelLR.predict_proba(X_test)[:,1]
X_test['predictions'] = pred_probability
X_test[['predictions']].to_csv("Predictions_tuned_test.csv", index=True)

In [245]:
X_test['predictions']

ID
106808150    0.066092
106810986    0.117239
106807679    0.120246
106816628    0.068239
106807602    0.033805
               ...   
106808018    0.072902
106805353    0.063059
106805295    0.351470
106806653    0.159744
106809200    0.152855
Name: predictions, Length: 2771, dtype: float64

In [247]:
X_test.drop(columns='predictions', inplace=True)

In [248]:
X_test.sample()

,AGE,GENDER,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,MARITAL_STATUS_1,...,JOB_DIR_8,JOB_DIR_9,JOB_DIR_10,WORK_TIME,FAMILY_INCOME_1,FAMILY_INCOME_2,FAMILY_INCOME_3,FAMILY_INCOME_4,FAMILY_INCOME_5,PERSONAL_INCOME
ID,,,,,,,,,,,,,,,,,,,,,
106817752,-0.446733,0.753345,1.151364,-0.526828,-0.650831,-0.197965,-0.128119,-0.009499,-0.035562,-0.561929,...,-0.123682,-0.026875,-0.066636,-0.012093,1.095282,-0.861839,-0.192486,-0.297816,-0.044596,-0.960666


In [249]:
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train, test_size=1/3, stratify=y_train)

In [251]:
pred_probability = modelLR.predict_proba(Xval)[:,1]

In [252]:
BestRec = -1
BestThr = -1
Acc_max = -1

for thr in np.arange(0, 1, 0.01):
    pred_class = pred_probability >= thr
    new_recall = recall_score(yval, pred_class)
    new_accuracy = accuracy_score(yval, pred_class)
    precision = precision_score(yval, pred_class)
    print(f'{thr=}, {new_recall=}, {precision=}')
    if new_accuracy >= Acc_max and new_recall > 0.66:
      BestRec = new_recall
      BestThr = thr
      Acc_max = new_accuracy
print (f'\nЛучший результат: {BestThr=}, {BestRec=}, {Acc_max=}')

thr=0.0, new_recall=1.0, precision=0.12719891745602166
thr=0.01, new_recall=0.997872340425532, precision=0.1273072747014115
thr=0.02, new_recall=0.997872340425532, precision=0.12817709756764142
thr=0.03, new_recall=0.9808510638297873, precision=0.12862723214285715
thr=0.04, new_recall=0.9744680851063829, precision=0.1311193816203836
thr=0.05, new_recall=0.951063829787234, precision=0.13403298350824588
thr=0.06, new_recall=0.9234042553191489, precision=0.13861386138613863
thr=0.07, new_recall=0.8808510638297873, precision=0.1423169474046064
thr=0.08, new_recall=0.8319148936170213, precision=0.14777021919879063
thr=0.09, new_recall=0.7787234042553192, precision=0.15410526315789475
thr=0.1, new_recall=0.7361702127659574, precision=0.16213683223992503
thr=0.11, new_recall=0.6851063829787234, precision=0.17037037037037037
thr=0.12, new_recall=0.6340425531914894, precision=0.18016928657799275
thr=0.13, new_recall=0.574468085106383, precision=0.18646408839779005
thr=0.14, new_recall=0.5106382

Лучший результат: BestThr=0.03, BestRec=0.6638297872340425, Acc_max=0.4912043301759134

In [256]:
pred_probability = modelLR.predict_proba(X_test)[:,1]

In [258]:
pred_probability

array([0.06609219, 0.11723865, 0.12024571, ..., 0.3514702 , 0.15974421,
       0.1528554 ])

In [257]:
get_metrics_score_thr(y_test, pred_probability, BestThr)

{'accuracy': 0.5416817033561891,
 'precision': 0.15949554896142434,
 'recall': 0.6107954545454546,
 'f1': 0.2529411764705882}

accuracy=0.43305665824612055, precision=0.13568439928272563, recall=0.6448863636363636, f1=0.22419753086419755

In [285]:
my_probibility = pd.read_csv('Predictions_tuned_test.csv', index_col='ID')

In [286]:
get_metrics_score_thr(y_test, my_probibility, BestThr)

{'accuracy': 0.5416817033561891,
 'precision': 0.15949554896142434,
 'recall': 0.6107954545454546,
 'f1': 0.2529411764705882}

## Бонус

Попробуйте применить другие модели классификации для решения этой задачи (любые какие знаете).

Удалось ли добиться улучшения качества модели?

In [ ]:
# Давайте оценим, что будет выдавать SGDClassifier на наших данных c параметрами по умолчанию

In [84]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score 

In [90]:
cross_val_score(SGDClassifier(), X_train, y_train, cv=3, scoring='roc_auc', verbose=True).mean()

0.5365779245547885

Убедился, что лучше качества c этой линейной моделью без тонкой настройки гиперпараметров не получится.

In [365]:
for param in gs_model_linear.get_params().keys():
    print(param)

cv
error_score
estimator__memory
estimator__steps
estimator__verbose
estimator__encoder
estimator__model
estimator__encoder__n_jobs
estimator__encoder__remainder
estimator__encoder__sparse_threshold
estimator__encoder__transformer_weights
estimator__encoder__transformers
estimator__encoder__verbose
estimator__encoder__verbose_feature_names_out
estimator__encoder__standardscaler
estimator__encoder__onehotencoder
estimator__encoder__standardscaler__copy
estimator__encoder__standardscaler__with_mean
estimator__encoder__standardscaler__with_std
estimator__encoder__onehotencoder__categories
estimator__encoder__onehotencoder__drop
estimator__encoder__onehotencoder__dtype
estimator__encoder__onehotencoder__feature_name_combiner
estimator__encoder__onehotencoder__handle_unknown
estimator__encoder__onehotencoder__max_categories
estimator__encoder__onehotencoder__min_frequency
estimator__encoder__onehotencoder__sparse
estimator__encoder__onehotencoder__sparse_output
estimator__model__C
estimator

In [298]:
y_test = pd.read_pickle('y_test.p')

13033    0.0
7958     0.0
4392     0.0
6484     0.0
12648    0.0
        ... 
10952    1.0
15596    0.0
1132     0.0
545      0.0
8642     0.0
Name: TARGET, Length: 2771, dtype: float64

In [300]:
y_test.to_csv("y_test.csv", index=False)

In [307]:
models = {
    'regular': {
        'rus_name': 'cтандартная',
        'model_type': 'логистическая регрессия',
        'params': 'полиномиальные признаки',
        'best_thr': 0.03,
    },
    'tuned': {
        'rus_name': 'настроенная',
        'model_type': 'логистическая регрессия',
        'params': 'С=1.2, max_iter=1000',
        'best_thr': 0.11,
    },
}

In [308]:
list(models.keys())

['regular', 'tuned']

In [309]:
t2 = zip(list(models), [1,2])

In [310]:
for models_names, col in t2:
    print(models[models_names])

regular 1
tuned 2
